In [1]:
import numpy as np
import pandas as pd
import random

In [12]:
supercell = dict.fromkeys(elements)
supercell

{'O': None, 'La': None, 'Zr': None, 'Li': None, 'Al': None}

In [13]:
atomic_positions_dict

{'O': array([[0.1       , 0.1956    , 0.2816    ],
        [0.89999998, 0.80439997, 0.7184    ],
        [0.40000001, 0.80439997, 0.7816    ],
        [0.60000002, 0.1956    , 0.2184    ],
        [0.89999998, 0.69560003, 0.2184    ],
        [0.1       , 0.3044    , 0.7816    ],
        [0.60000002, 0.3044    , 0.7184    ],
        [0.40000001, 0.69560003, 0.2816    ],
        [0.2816    , 0.1       , 0.1956    ],
        [0.7184    , 0.89999998, 0.80439997],
        [0.7816    , 0.40000001, 0.80439997],
        [0.2184    , 0.60000002, 0.1956    ],
        [0.2184    , 0.89999998, 0.69560003],
        [0.7816    , 0.1       , 0.3044    ],
        [0.7184    , 0.60000002, 0.3044    ],
        [0.2816    , 0.40000001, 0.69560003],
        [0.1956    , 0.2816    , 0.1       ],
        [0.80439997, 0.7184    , 0.89999998],
        [0.80439997, 0.7816    , 0.40000001],
        [0.1956    , 0.2184    , 0.60000002],
        [0.69560003, 0.2184    , 0.89999998],
        [0.3044    , 0.7816  

In [5]:
name = 'LLZO_Ia3d'
old_name = name + "_filled"
new_name = name + "_Al10_partial_test"

# not relevant
occupations = {'O': 1.0,
              'La': 0.985,
              'Zr': 1.0,
              'Li1': 0.69,
              'Al': 0.072,
              'Li2': 0.32}
# relevant
LI1_sites = 24
LI2_sites = 96
LI1_occupy = 21
LI2_occupy = 32
AL_occupy = 1

supercell = 2

In [6]:
with open(old_name+'.vasp', 'r') as f:
    lines = f.readlines()

scale = lines[1]
lattice = lines[2:5]
composition = lines[5:7]
method = lines[7]
atomic_positions = lines[8:]

In [7]:
elements = np.array(composition[0].rstrip('\n').split(' '))
elements = elements[elements != '']
numbers = np.array(composition[1].rstrip('\n').split(' '))
numbers = numbers[numbers != '']
composition_dict = {elements[i]: numbers[i] for i in range(len(elements))}

In [8]:
atomic_positions_conv = np.array([x.rstrip('\n').split(" ") for x in atomic_positions])
atomic_positions_conv = np.array([x[x != ''] for x in atomic_positions_conv])
atomic_positions_conv = atomic_positions_conv.astype('float')

In [9]:
atomic_positions_dict = dict.fromkeys(elements)
s = 0
for element in elements:
    atomic_positions_dict[element] = atomic_positions_conv[s:s + int(composition_dict.get(element))]
    s = s + int(composition_dict.get(element))

In [208]:
octahedral_Li = atomic_positions_dict['Li'][LI1_sites:]
octahedral_Li_sort = np.empty((0,3))
while True:
    if octahedral_Li.size == 0:
        break
    v1 = octahedral_Li[0]
    min_dist = 1
    min_v2 = np.array([0,0,0])
    min_j = 1
    for j in np.arange(1, len(octahedral_Li)):
        v2 = octahedral_Li[j]
        dist = np.linalg.norm(v1-v2)
        if dist<min_dist:
            min_dist = dist
            min_v2 = v2.copy()
            min_j = j
    octahedral_Li_sort = np.append(octahedral_Li_sort, [v1], axis=0)
    octahedral_Li_sort = np.append(octahedral_Li_sort, [min_v2], axis=0)
    octahedral_Li = np.delete(octahedral_Li, [min_j,0], 0)

- O:   96/96 O
- Zr:  16/16 Zr
- La:  24/24 La
- Li2: 32/96 Li, 64/96 Vac.
- Li1: 18/24 Li,  2/24 Al,  4/24 Vac.

In [248]:
new_atomic_positions_dict = dict.fromkeys(elements)
new_atomic_positions_dict['O'] = atomic_positions_dict['O']
new_atomic_positions_dict['Zr'] = atomic_positions_dict['Zr']
new_atomic_positions_dict['La'] = atomic_positions_dict['La']

lithiums = np.empty((0,3))
aluminums = np.empty((0,3))

# Octahedral lithium filling
indices = np.arange(len(octahedral_Li_sort)//2)
np.random.shuffle(indices)
indices = indices[:LI2_occupy]
for index in indices:
    num = int(round(random.uniform(0,1), 0))
    lithiums = np.append(lithiums, [octahedral_Li_sort[2*index + num]], axis=0)

# Tetragonal site filling
indices = np.arange(24)
np.random.shuffle(indices)
for index in indices[:LI1_occupy]:
    lithiums = np.append(lithiums, [atomic_positions_dict['Li'][index]], axis=0)

new_atomic_positions_dict['Li'] = lithiums

for index in indices[LI1_occupy:LI1_occupy+AL_occupy]:
    aluminums = np.append(aluminums, [atomic_positions_dict['Al'][index]], axis=0)
new_atomic_positions_dict['Al'] = aluminums

In [268]:
new_composition_dict = {}
for key, value in new_atomic_positions_dict.items():
    new_composition_dict[key] = value.shape[0]
elements = list(new_composition_dict.keys())
numbers = list(new_composition_dict.values())

In [275]:
new_atomic_positions_dict

{'O': array([[0.1       , 0.1956    , 0.2816    ],
        [0.89999998, 0.80439997, 0.7184    ],
        [0.40000001, 0.80439997, 0.7816    ],
        [0.60000002, 0.1956    , 0.2184    ],
        [0.89999998, 0.69560003, 0.2184    ],
        [0.1       , 0.3044    , 0.7816    ],
        [0.60000002, 0.3044    , 0.7184    ],
        [0.40000001, 0.69560003, 0.2816    ],
        [0.2816    , 0.1       , 0.1956    ],
        [0.7184    , 0.89999998, 0.80439997],
        [0.7816    , 0.40000001, 0.80439997],
        [0.2184    , 0.60000002, 0.1956    ],
        [0.2184    , 0.89999998, 0.69560003],
        [0.7816    , 0.1       , 0.3044    ],
        [0.7184    , 0.60000002, 0.3044    ],
        [0.2816    , 0.40000001, 0.69560003],
        [0.1956    , 0.2816    , 0.1       ],
        [0.80439997, 0.7184    , 0.89999998],
        [0.80439997, 0.7816    , 0.40000001],
        [0.1956    , 0.2184    , 0.60000002],
        [0.69560003, 0.2184    , 0.89999998],
        [0.3044    , 0.7816  

In [277]:
with open(new_name+".vasp", "w") as f:
    f.write(new_name+'\n')
    f.write(scale)
    f.write(lattice[0])
    f.write(lattice[1])
    f.write(lattice[2])
    f.write(" ".join(elements)+'\n')
    f.write(" ".join(np.array(numbers).astype('str'))+'\n')
    f.write(method)
    for key, value in new_atomic_positions_dict.items():
        rows = [' '.join(str(x) for x in line) for line in value]
        f.write('\n'.join(rows))
        f.write('\n')